In [10]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')



In [20]:
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()
# spmc_zoomout
driver.implicitly_wait(10)
#         a:nth-child(3)#

search_box = driver.find_element_by_css_selector("input#search-input")
search_box.send_keys("망리단길")

search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)
search_box.clear()

#지도레벨 축소
# for i in range(3):    
#     driver.find_elements_by_css_selector("a.spmc_zoomout")[0].click()
#     time.sleep(0.25)


#print(chrome_ver)

## store ids 
store_ids = [];

In [18]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [21]:
search_query  = ['식당']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 50;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break
 

1452486891
629167710
21547944
1320083638
15113951
15113989
1133618933
15113376
36004379
33300088
1936370097
1780379225
15113586
1056034552
1281401007
35014209
15115003
11801364
15114840
1959332974
15114787
11070241
1197072667
15115665
37081616
1250575806
1853831598
1102312410
1345231656
36524355
37774691
32045663
1782678732
1400690138
15114830
13495681
12076089
15115332
38767337
1724870570
15115399
12998617
15114363
15115394
20009574
36505420
824527947
160828633
15116102
15085364
15100843
1639583821
1165562808
20273805
1975226345
1446769333
15379107
12012763
15115378
15114726
1699874050
15114744
33877589
510742091
37294279
1086132747
1416630486
15115504
15115054
1217725976
15115337
15115246
1090690153
1229956430
1423679422
1012080500
15090810
1579722717
1591279841
15379294
1795212691
1149997082
12001256
15102303
12801773
32048894
1318552498
15079613
1848932798
1653039582
1823619000
36369949
31108108
1387914832
1945404929
1241609622
37906802
11831368
1127698430
37039697
1076191439
10376

KeyboardInterrupt: 

In [22]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)
        
        try:
            for i in range(20):
                more_btn = driver.find_element_by_class_name("_3iTUo")
                if(more_btn):
                    more_btn.click()
                    time.sleep(0.5)
                else:
                    break;
        except Exception as ex:
            print("error catch : " , ex)
        
        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    #3점 ~ 4점대 리뷰는 거른다 (3점미만 : 부정 , 5점 : 긍정)
                    if(float(rating_ll.text) < 3 or float(rating_ll.text) > 4.5):
                        review_content = review_ll.text
                        label = "0" #0 : 긍정, 1 부정
                        if("\n" in review_ll.text):
                            review_content = review_ll.text.replace("\n"," ")
                            
                        #3점 미만이면 부정평가
                        if(float(rating_ll.text) < 3 ):
                            label = "1";
                            
                            with open("review_data.txt", "a") as file:
                                # id , 별점, 리뷰내용, 레이블(0:긍정, 1:부정)
                                file.write(store_id+"\t" + rating_ll.text+"\t" + review_content +"\t"+ label+ "\n") 
    #                     print("rating : " + rating_ll.text)
    #                     print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 193024519
id : 1940486041
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 725917377
id : 37797967
id : 11672261
id : 33880194
id : 20601610
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 32516913
id : 11672174
id : 37611484
id : 20601620
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 33102171
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1420051282
error catch :  Message: no such element: U

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1367200281
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 13572480
id : 36369460
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1470424364
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1813424354
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36893616
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91

id : 34485554
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1238516524
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 34959663
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1549718128
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36626866
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1812650760
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 31824990
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 35545556
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1209402917
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 33130016
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 37203095
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id :

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1426838974
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1106199165
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 13012964
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 38217720
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1592471561
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)



error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 495320744
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 38555426
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1773964786
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1978995641
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1955391306
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)


error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 11015660531452486891
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 629167710
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 21547944
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1320083638
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15113951
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.44

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36505420
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 824527947
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 160828633
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15116102
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15085364
error catch :  Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.101)

id : 15100843
error catc

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36369949
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 31108108
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1387914832
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1945404929
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1241609622
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)



error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1626314303
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1056924136
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1779749409
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 12869573
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1127879734
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)